In [101]:
import pandas as pd
import numpy as np
import requests
import os

# Gather

Code for downloading Udacity's Dog Prediction Data

In [12]:
#url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
#r = requests.get('https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv')

In [16]:
# make a directory if one does not already exist
#folder_name = 'dog_predictions'
#if not os.path.exists(folder_name):
    #os.makedirs(folder_name)

In [23]:
#with open(os.path.join(folder_name, 
                           #url.split('/')[-1]), mode='wb') as file:
             #file.write(r.content)

Querying Retweets and Favorites for Relevant Tweets from Twitter API

Initializing All Relevant Datasets

In [25]:
twitter_archive = pd.read_csv('twitter-archive-enhanced.csv')
twitter_archive;

In [26]:
predicts = pd.read_csv('dog_predictions/image-predictions.tsv', sep='\t')
predicts;

# Assess

### Documented Issues

`twitter_archive`
#### Quality
- timestamp is a string and not datetime
- source contains irrelevant variable, variable is found in text column
- missing values in expanded_urls
- There are 41 instances where categorical variables are found in the text, but are not accounted for
- name column is not always accurate, (ex: index 542 name is considered "incredibly" since text before contains "incredible"
    -possible solution is to examine all names that are not upper-case


#### Tidiness
- text column contains a source variable for the tweet
- dog "ages/types" (floofer, pupper etc.) should be single, categorial column

`predicts`
#### Quality


#### Tidiness


`retweet`
#### Quality


#### Tidiness


In [48]:
pd.set_option('display.max_colwidth', -1)

### twitter_archive

In [299]:
#to be used for visual assessments. Supressed to save space.
twitter_archive.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
542,806219024703037440,NaN,NaN,2016-12-06 19:29:28 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",We only rate dogs. Please stop sending in non-canines like this Freudian Poof Lion. This is incredibly frustrating... 11/10 https://t.co/IZidSrBvhi,NaN,NaN,NaN,https://twitter.com/dog_rates/status/806219024703037440/photo/1,11,10,incredibly,None,None,None,None
117,870063196459192321,NaN,NaN,2017-05-31 23:43:25 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Meet Clifford. He's quite large. Also red. Good w kids. Somehow never steps on them. Massive poops very inconvenient. Still 14/10 would ride https://t.co/apVOyDgOju,NaN,NaN,NaN,"https://twitter.com/dog_rates/status/870063196459192321/photo/1,https://twitter.com/dog_rates/status/870063196459192321/photo/1",14,10,Clifford,None,None,None,None
1632,684460069371654144,NaN,NaN,2016-01-05 19:42:51 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Jeph. He's a Western Sagittarius Dookmarriot. Frightened by leaf. Caught him off guard. 10/10 calm down Jeph https://t.co/bicyOV6lju,NaN,NaN,NaN,https://twitter.com/dog_rates/status/684460069371654144/photo/1,10,10,Jeph,None,None,None,None
102,872620804844003328,NaN,NaN,2017-06-08 01:06:27 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Monkey. She's supporting owners everywhere with her fancy #PrideMonth bandana. 13/10 love is love is love... https://t.co/lUcpnZDPz9,NaN,NaN,NaN,https://twitter.com/dog_rates/status/872620804844003328/photo/1,13,10,Monkey,None,None,None,None
416,822610361945911296,NaN,NaN,2017-01-21 01:02:48 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Please stop sending in non-canines like this Very Pettable Dozing Bath Tortoise. We only rate dogs. Only send dogs... 12/10 https://t.co/mcagPeENIh,NaN,NaN,NaN,https://twitter.com/dog_rates/status/822610361945911296/photo/1,12,10,None,None,None,None,None


In [59]:
#no duplicated values
list(twitter_archive.duplicated()).count(True)

0

In [208]:
#to be used for visual assessments. commented out to save space
#twitter_archive.info();

In [209]:
#investigating max values for numerator/denominator
twitter_archive.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


In [315]:
def category_accuracy(df, columns_list, known_values_list):
    '''category_accuracy parses through the texts column of a dataframe and checks if the information matches
    the values of the categorical columns, GIVEN that the text is used as the categorical variable source
    and records are single, known values.
    Restrictions: text column must be called "text".
    Returns: category_accuracy returns the index of rows where the categorical values do not match the text'''
    offending_rows = []
    # matching text with other columns
    for index, row in df.iterrows():
        match = []
        secondary = [row[column] for column in columns_list]
        source = row.text.split()
        for word in source:
            match += [word for value in known_values_list if value in word and word != 'None']
    #checking accuracy
        if len(match)> 0 and all(current == 'None' for current in secondary) == True:
            offending_rows.append(index)
    return offending_rows

### Parsing Text for Accuracy

In [316]:
#category_accuracy(df, columns, known values list)
offending_rows = category_accuracy(twitter_archive, ['doggo', 'floofer', 'pupper', 'puppo'], ['doggo', 'floofer', 'pupper', 'puppo'])
print('The number of instances where doggo, floofer, pupper, and puppo is found in the text, but does not have a value is: ', len(offending_rows))

The number of instances where doggo, floofer, pupper, and puppo is found in the text, but does not have a value is:  41


In [318]:
#investigate offending rows. Rows had issues. Code has been commented out to save space.
#for i in offending_rows:
    #display(twitter_archive[twitter_archive.index == i])

True

### Investigating Max and Min Values

In [56]:
twitter_archive.rating_numerator.value_counts();

In [50]:
#non-issue. 1776 is the year the DoI was signed
twitter_archive[twitter_archive.rating_numerator == 1776]['text']

979    This is Atticus. He's quite simply America af. 1776/10 https://t.co/GRXwMxLBkh
Name: text, dtype: object

In [55]:
twitter_archive.rating_denominator.value_counts();

In [49]:
#non-issue. 170 matches the actual post
twitter_archive[twitter_archive.rating_denominator == 170]['expanded_urls']

1120    https://twitter.com/dog_rates/status/731156023742988288/photo/1
Name: expanded_urls, dtype: object

## predicts

# Clean

Creating Cleaning Copies

In [27]:
twitter_archive_clean = twitter_archive.copy()
predicts_clean = predicts.copy()

# Analysis

(jotting ideas down early so as to not forget)
### Motivating Questions:
- Do higher "ratings" correlate with higher number of retweets? (Must define ratings. Std might be useful)
- Investigate which is more "popular": Cute or Funny. Cute defined by Kindchenschema and funny defined by Benign Violation.
- Within the categories of cuteness and funniness, are more extreme examples more popular? Measured by retweets.
- What are observed characteristics of the top 3 most popular tweets? Is there a theme?

Although the ratings individually do not seem to make much sense due to the numerator exceeding the denominator, perhaps the ratings can be better understood as a decimal score, with higher scores indicating higher approval.